In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import gmaps.datasets
import requests

# Bike Data & gmaps api key
from config import api_key

In [ ]:
gmaps.configure(api_key=)